In [137]:
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv")
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/promoted_content.csv")
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/sample_submission.csv")
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_topics.csv")
clicks_train = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : object, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views_sample.csv")

In [138]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [139]:
#split to validation and train
leng = len(clicks_train.display_id.unique())
start_display_id_validation = (4 * leng / 5)
start_index_validation = clicks_train[clicks_train['display_id'] == start_display_id_validation].index.tolist()[0]
validation_labels = clicks_train[start_index_validation:]['clicked']
validation = clicks_train[start_index_validation:].drop('clicked',axis = 1)
train_labels = clicks_train[:start_index_validation]['clicked']
train = clicks_train[:start_index_validation].drop('clicked',axis = 1)
del clicks_train

In [141]:
print validation_labels.head()
print validation.head()
print train_labels.head()
print train.head()

69673432    False
69673433     True
69673434    False
69673435    False
69673436    False
Name: clicked, dtype: bool
          display_id   ad_id  document_id
69673432    13499674  169725      2663860
69673433    13499674  475956      2663860
69673434    13499674  527161      2663860
69673435    13499674  528483      2663860
69673436    13499675  173403      1157326
0    False
1    False
2     True
3    False
4    False
Name: clicked, dtype: bool
   display_id   ad_id  document_id
0           1   42337       379743
1           1  139684       379743
2           1  144739       379743
3           1  156824       379743
4           1  279295       379743


In [142]:
#remove all confidence levels below 0.3, can decide on another criteria later on
#documents_entities = documents_entities[documents_entities.confidence_level > 0.3]
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
documents_topics = documents_topics[documents_topics.confidence_level > 0.3]

In [143]:
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
documents_topics.drop('confidence_level',axis=1, inplace=True)

In [144]:
#prepare one-hot
topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")

In [145]:
topics = documents_topics.join(topic_dummies)
categories = documents_categories.join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index().drop('category_id',1)
topics = topics.groupby(['document_id']).sum().reset_index().drop('topic_id',1)
del documents_categories, documents_topics, categories_dummies, topic_dummies

In [149]:
print topics.head(), categories.head()

   document_id  _ti_0  _ti_1  _ti_2  _ti_3  _ti_4  _ti_5  _ti_6  _ti_7  _ti_8  \
0          157    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1          159    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2          161    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          650    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4          664    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

    ...     _ti_290  _ti_291  _ti_292  _ti_293  _ti_294  _ti_295  _ti_296  \
0   ...         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1   ...         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2   ...         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3   ...         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4   ...         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   _ti_297  _ti_298  _ti_299  
0      0.0      0.0

In [175]:
#create promoted, promoted content (ad information) with its' categories and topics
promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id')
promoted = promoted.merge(topics, how='left', on='document_id').drop('document_id', axis=1)
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [179]:
#promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/promoted_content.csv")
print promoted.head()

   ad_id  _ci_1000  _ci_1100  _ci_1200  _ci_1202  _ci_1203  _ci_1204  \
0      1       0.0       0.0       0.0       0.0       0.0       0.0   
1      2       0.0       0.0       0.0       0.0       0.0       0.0   
2      3       0.0       0.0       0.0       0.0       0.0       0.0   
3      4       0.0       0.0       0.0       0.0       0.0       0.0   
4      5       0.0       0.0       0.0       0.0       0.0       0.0   

   _ci_1205  _ci_1206  _ci_1207   ...     _ti_290  _ti_291  _ti_292  _ti_293  \
0       0.0       0.0       0.0   ...         0.0      0.0      0.0      0.0   
1       0.0       0.0       0.0   ...         0.0      0.0      0.0      0.0   
2       0.0       0.0       0.0   ...         0.0      0.0      0.0      0.0   
3       1.0       0.0       0.0   ...         0.0      0.0      0.0      0.0   
4       0.0       0.0       0.0   ...         0.0      0.0      0.0      0.0   

   _ti_294  _ti_295  _ti_296  _ti_297  _ti_298  _ti_299  
0      0.0      0.0      0.0

In [182]:
#divide to chunks
chunk_size = float(train.shape[0])/82
print chunk_size
chunk_size = int(chunk_size)

849676


In [24]:
for i in range(82):
    start = i * chunk_size
    end = (i+1) * chunk_size
    train_chunk = train[start:end].merge(promoted,how = 'left',on = 'ad_id')
    train_chunk = train_chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    train_chunk = train_chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    train_chunk.fillna(0.0, inplace=True)
    train_chunk

In [183]:
start = 0 * chunk_size
end = (0+1) * chunk_size
train_chunk = train[start:end].merge(promoted,how = 'left',on = 'ad_id')
train_chunk = train_chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
train_chunk = train_chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))

In [185]:
print train_chunk.describe

<bound method DataFrame.describe of         display_id   ad_id  document_id  _ci_1000_ad  _ci_1100_ad  \
0                1   42337       379743          0.0          0.0   
1                1  139684       379743          0.0          0.0   
2                1  144739       379743          0.0          0.0   
3                1  156824       379743          0.0          0.0   
4                1  279295       379743          0.0          0.0   
5                1  296965       379743          0.0          0.0   
6                2  125211      1794259          0.0          0.0   
7                2  156535      1794259          0.0          0.0   
8                2  169564      1794259          0.0          0.0   
9                2  308455      1794259          0.0          0.0   
10               3   71547      1179111          0.0          0.0   
11               3   95814      1179111          0.0          0.0   
12               3  152141      1179111          0.0          0.0  